In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
from multiprocessing import Pool
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
documents = pd.read_csv('../DATA/DATA.csv')

In [3]:
documents

,INDEX,TEXT
0,A001-00,Auto 001/00\n\nDEBIDO PROCESO DE TUTELA-Notifi...
1,A001-01,Auto 001/01\n\nNULIDAD SENTENCIA DE CONSTITUCI...
2,A001-02,Auto 001/02\n \nLEGITIMACION POR PASIVA EN TUT...
3,A001-03,Auto 001/03\n\nREFERENDO-Etapas de desarrollo ...
4,A001-04,Auto 001/04\n\nCONFLICTO DE COMPETENCIA EN TUT...
...,...,...
35732,T-999-08,Sentencia T-999/08\n\nDERECHO A LA SALUD COMO ...
35733,T-999-10,Sentencia T-999/10\n\nMUJER EMBARAZADA-Deber d...
35734,T-999-12,Sentencia T-999/12\n\n\nACCION DE TUTELA DE PE...
35735,T-999-99,Sentencia T-999/99\n\nDERECHO A LA SEGURIDAD S...


In [4]:
def preprocessor_sentences(text):
    """
    preprocesador de texto con separación de frases por línea.
    """
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.replace("\x93", " ")
    text = text.replace("\x94", " ")
    text = text.replace("\r", " ")
    text = text.replace("\t", " ")


    return text

In [5]:
documents["TEXT_PLAIN"] = [preprocessor_sentences(str(x)) for x in tqdm(documents.TEXT)]

100%|██████████| 35737/35737 [00:02<00:00, 13800.52it/s]


In [6]:
documents

,INDEX,TEXT,TEXT_PLAIN
0,A001-00,Auto 001/00\n\nDEBIDO PROCESO DE TUTELA-Notifi...,Auto 001/00 DEBIDO PROCESO DE TUTELA-Notifica...
1,A001-01,Auto 001/01\n\nNULIDAD SENTENCIA DE CONSTITUCI...,Auto 001/01 NULIDAD SENTENCIA DE CONSTITUCION...
2,A001-02,Auto 001/02\n \nLEGITIMACION POR PASIVA EN TUT...,Auto 001/02 LEGITIMACION POR PASIVA EN TUTEL...
3,A001-03,Auto 001/03\n\nREFERENDO-Etapas de desarrollo ...,Auto 001/03 REFERENDO-Etapas de desarrollo de...
4,A001-04,Auto 001/04\n\nCONFLICTO DE COMPETENCIA EN TUT...,Auto 001/04 CONFLICTO DE COMPETENCIA EN TUTEL...
...,...,...,...
35732,T-999-08,Sentencia T-999/08\n\nDERECHO A LA SALUD COMO ...,Sentencia T-999/08 DERECHO A LA SALUD COMO GA...
35733,T-999-10,Sentencia T-999/10\n\nMUJER EMBARAZADA-Deber d...,Sentencia T-999/10 MUJER EMBARAZADA-Deber del...
35734,T-999-12,Sentencia T-999/12\n\n\nACCION DE TUTELA DE PE...,Sentencia T-999/12 ACCION DE TUTELA DE PERSO...
35735,T-999-99,Sentencia T-999/99\n\nDERECHO A LA SEGURIDAD S...,Sentencia T-999/99 DERECHO A LA SEGURIDAD SOC...


In [7]:
documents['INDEX'] = documents['INDEX'].str.upper()

In [8]:
documents["DOCUMENT_TYPE"] = [x.strip()[0] for x in documents.INDEX]

In [9]:
documents.columns.tolist()

['INDEX', 'TEXT', 'TEXT_PLAIN', 'DOCUMENT_TYPE']

In [10]:
documents = documents[['INDEX','DOCUMENT_TYPE', 'TEXT', 'TEXT_PLAIN']]

# ARCHIVO DE PRUEBA

In [12]:
ref = 0
test = documents.iloc[ref].TEXT_PLAIN
print(documents.iloc[ref].TEXT)

Auto 001/00

DEBIDO PROCESO DE TUTELA-Notificación iniciación de la acción

Si bien el trámite propio de la acción de tutela es breve, sumario e informal, toda vez que busca la protección inmediata de un derecho fundamental cuando quiera que éste se vea amenazado o vulnerado por la acción u omisión de una autoridad pública o de un particular en los casos expresamente señalados, no puede llevarse a cabo el proceso sin el conocimiento de la autoridad o del particular contra quien se impetró, pues de lo contrario se le estaría vulnerando en forma flagrante su derecho a la defensa y por ende al debido proceso.



Referencia: Expediente T-246504

Acción de tutela instaurada por Lorenza Díaz Padilla contra el Hospital San Jerónimo de Montería

Magistrado Sustanciador:
Dr. JOSÉ GREGORIO HERNÁNDEZ GALINDO

Santafé de Bogotá D.C., doce (12) de enero de dos mil (2000).

I. ANTECEDENTES.

La accionante manifiesta que es pensionada del hospital San Jerónimo de Montería, entidad que le adeuda las m

In [78]:
document_target = 'SU123-18'
documents.index[documents['INDEX'] == document_target].tolist()

[16041]

In [79]:
ref = documents.index[documents['INDEX'] == document_target].tolist()[0]
test = documents.iloc[ref].TEXT_PLAIN
print(documents.iloc[ref].TEXT)

Sentencia SU123/18


COMUNIDADES INDIGENAS-Legitimación por activa en los casos que reclaman protección de sus derechos fundamentales por medio de acción de tutela
ACCION DE TUTELA PARA LA PROTECCION DEL DERECHO A LA CONSULTA PREVIA-Único mecanismo judicial eficaz para garantizar que los pueblos indígenas sean consultados 
La Sala considera que los medios de control de nulidad simple, así como de nulidad y restablecimiento del derecho, no son mecanismos judiciales idóneos para proteger el derecho de la consulta previa del cabildo demandante, perteneciente a la etnia Awá. El Gobernador del cabildo indígena formuló argumentos plausibles que podrían evidenciar una omisión en el trámite de consulta previa, procedimiento obligatorio para los proyectos de explotación de hidrocarburos que tienen la virtualidad de causar una afectación directa a la comunidad.
PRINCIPIO DE INMEDIATEZ EN ACCION DE TUTELA-Reglas generales
DERECHO A LA CONSULTA PREVIA DE COMUNIDADES ETNICAS-Naturaleza y alcance 
D

# Resumen con textrank (Aprobado)
https://radimrehurek.com/gensim_3.8.3/summarization/summariser.html

In [13]:
from gensim.summarization import summarize

In [80]:
print((summarize(test)))

Sentencia SU123/18   COMUNIDADES INDIGENAS-Legitimación por activa en los casos que reclaman protección de sus derechos fundamentales por medio de acción de tutela ACCION DE TUTELA PARA LA PROTECCION DEL DERECHO A LA CONSULTA PREVIA-Único mecanismo judicial eficaz para garantizar que los pueblos indígenas sean consultados  La Sala considera que los medios de control de nulidad simple, así como de nulidad y restablecimiento del derecho, no son mecanismos judiciales idóneos para proteger el derecho de la consulta previa del cabildo demandante, perteneciente a la etnia Awá.
PRINCIPIO DE INMEDIATEZ EN ACCION DE TUTELA-Reglas generales DERECHO A LA CONSULTA PREVIA DE COMUNIDADES ETNICAS-Naturaleza y alcance  DERECHO A LA CONSULTA PREVIA DE COMUNIDADES ETNICAS-Protección constitucional e internacional   CONVENIO 169 DE LA OIT Y BLOQUE DE CONSTITUCIONALIDAD-Fundamento del derecho a la consulta previa   DERECHO FUNDAMENTAL A LA CONSULTA PREVIA-Ámbito de aplicación y contenido CONSULTA PREVIA-A

# SUMY 
### machinelearningplus.com/nlp/text-summarization-approaches-nlp-example/
https://github.com/miso-belica/sumy

In [15]:
import sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer

In [16]:
from sumy.summarizers.lex_rank import LexRankSummarizer

In [81]:
my_parser = PlaintextParser.from_string(test,Tokenizer('spanish'))

## LEX RANK (Aprobado)

In [82]:
# Creating a summary of 3 sentences.
lex_rank_summarizer = LexRankSummarizer()
lexrank_summary = lex_rank_summarizer(my_parser.document,sentences_count=3)

# Printing the summary
for sentence in lexrank_summary:
    print(sentence)
    print('-'*100)

CONSULTA PREVIA Y CONSENTIMIENTO PREVIO, LIBRE E INFORMADO-Deben entenderse en el marco del principio de proporcionalidad y razonabilidad de las medidas DERECHO A LA CONSULTA PREVIA Y DERECHO AL CONSENTIMIENTO LIBRE E INFORMADO-Diferencias  DERECHO A LA CONSULTA PREVIA-Deber estatal de respetar y garantizar el derecho CERTIFICADOS DE PRESENCIA DE COMUNIDADES ETNICAS EXPEDIDOS POR LA DIRECCION DE CONSULTA PREVIA DEL MINISTERIO DEL INTERIOR-Dificultades  DERECHO A LA CONSULTA PREVIA-Alcance de la certificación de la presencia de comunidades indígenas proferida por el Ministerio del Interior DERECHO A LA CONSULTA PREVIA-Utilización del estándar de debida diligencia de las empresas El Relator Especial para los derechos humanos y las libertades de los pueblos indígenas, ha señalado el deber que tienen los Estados y las empresas de seguir los parámetros mínimos de debida diligencia para la garantía de los derechos de las comunidades indígenas, en particular, la consulta previa.
-------------

## LSA (Latent semantic analysis)

In [60]:
from sumy.summarizers.lsa import LsaSummarizer

In [83]:
parser = PlaintextParser.from_string(test,Tokenizer('spanish'))

In [84]:
lsa_summarizer=LsaSummarizer()
lsa_summary= lsa_summarizer(parser.document,3)

# Printing the summary
for sentence in lsa_summary:
    print(sentence)
    print('-'*100)

El segundo consistirá en proponer e implementar las medidas requeridas para prevenir, mitigar, corregir, recuperar o restaurar los efectos de esa actividad extractiva.
----------------------------------------------------------------------------------------------------
Dichos impactos se exacerban debido al estrecho vínculo que une a éstos con sus territorios para su supervivencia como grupos étnicos.
----------------------------------------------------------------------------------------------------
Así mismo por la Corte Interamericana de Derechos Humanos, Sentencia (excepciones preliminares, fondo, reparaciones y costas) Caso del Pueblo Saramaka vs.
----------------------------------------------------------------------------------------------------


## Luhn (Aprobado)

In [63]:
from sumy.summarizers.luhn import LuhnSummarizer

In [85]:
luhn_summarizer=LuhnSummarizer()
luhn_summary=luhn_summarizer(parser.document,sentences_count=3)

# Printing the summary
for sentence in luhn_summary:
    print(sentence)
    print('-'*100)

A la hora de determinar el remedio judicial correspondiente y valorar las actuaciones que se desarrollan en el marco de la consulta previa, solo se pueden permitir limitaciones que sean constitucionalmente legítimas, es decir, que se trate de una medida necesaria para salvaguardar un interés de superior jerarquía y que se trate de la medida menos gravosa para la autonomía que se les reconoce a las comunidades étnicas.148   Para el efecto se deben ponderar y explorar los siguientes elementos: (i) la posición y las propuestas de los actores, (ii) el comportamiento de la empresa y en particular si ésta tuvo o no la debida diligencia frente a los derechos del grupo étnico,  (iii) cuál es la mejor forma de garantizar los derechos fundamentales de los miembros de los pueblos indígenas y de los demás habitantes de los respectivos territorios -tales como el derecho a la vida e integridad personal, al libre desarrollo de la personalidad, a la seguridad y a la salud-, (iv) la protección del inte

### KLSummarizer

In [65]:
from sumy.summarizers.kl import KLSummarizer

In [86]:
kl_summarizer=KLSummarizer()
kl_summary=kl_summarizer(parser.document,sentences_count=3)

# Printing the summary
for sentence in kl_summary:
    print(sentence)
    print('-'*100)

KeyboardInterrupt: 

# SPACY

In [39]:
import spacy
import pytextrank

In [44]:
nlp = spacy.load('es_core_news_lg')

In [47]:
nlp.add_pipe("textrank")

In [87]:
doc = nlp(test)

## TEXT RANK 
https://pypi.org/project/pytextrank/

In [88]:
for phrase in doc._.phrases:
    print(phrase.text)
    print(phrase.rank, phrase.count)
    print('-'*50)

COMUNIDADES
0.06444127815297 2
--------------------------------------------------
comunidad
0.06444127815297 13
--------------------------------------------------
comunidades
0.06444127815297 61
--------------------------------------------------
derecho
0.04951625165679734 69
--------------------------------------------------
derechos
0.04951625165679734 13
--------------------------------------------------
CONSULTA
0.04809936463901187 10
--------------------------------------------------
Consulta
0.04809936463901187 2
--------------------------------------------------
consulta
0.04809936463901187 54
--------------------------------------------------
consultas
0.04809936463901187 2
--------------------------------------------------
PROYECTOS
0.047331423569080475 2
--------------------------------------------------
proyecto
0.047331423569080475 147
--------------------------------------------------
proyectos
0.047331423569080475 22
--------------------------------------------------
CONS

## TEXT RANK
https://towardsdatascience.com/textrank-for-keyword-extraction-by-python-c0bae21bcec0

In [33]:
from collections import OrderedDict
from spacy.lang.es.stop_words import STOP_WORDS

In [34]:
class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [89]:
tr4w = TextRank4Keyword()
tr4w.analyze(test, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(20)

comunidad - 28.67870137138442
consulta - 22.183123974636164
Awá - 19.504027866829603
proyecto - 17.03286346066432
derechos - 16.68628322955011
comunidades - 16.136460186229243
derecho - 15.885012233331818
Corte - 15.124246856321953
territorio - 12.165882724829007
afectación - 11.98260957104964
área - 11.625059079222563
pueblos - 11.04213733400792
Cabaña - 10.793250288751574
explotación - 10.500879848956986
caso - 10.113326048445648
actividades - 10.088610618468254
pueblo - 9.599691634781216
Colombia - 9.549075125306869
presencia - 9.190350991940766
Ministerio - 8.820918413608238
relación - 7.799163336942349
ii - 7.727499317319196
